Train Model using  NetFlow files from csv format

Import dependecies
pip install numpy,pandas,xgboost,sklearn,pickle

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier,LocalOutlierFactor
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.cluster import DBSCAN,KMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import model_selection
from pickle import dump
import time
from pickle import load

Data set can be downloaded from here

https://rdm.uq.edu.au/files/650f1fa0-ef9c-11ed-b5f6-b1a04f482c13

Columns needed from csv files
Example:
172.31.66.17,51128,23.36.69.189,443,6,91.0,152,0,3,0,194,4285680,0,anomaly_name

After data normalization some field will be dropped
'src_ip', 'dst_ip','l7_proto','anomaly'
In this way these values can be written randomly and will be not used in learning process

field 'label' if equels 0 - is normal, if equals 1 - is anomaly, training model dont take it in to cosideration, just needed for validation

In [3]:
flow_fields = [
    "src_ip",
    "src_port",
    "dst_ip",
    "dst_port",
    "ip_protocol",
    "l7_proto",
    "in_bytes",
    "out_bytes",
    "in_pkts",
    "out_pkts",
    "tcp_flags",
    "duration",
    "label",
    "anomaly",
    "notneeded"
]

with open("datasets/NF-UQ-NIDS.csv", "r") as csvfile:
    # pass input data stream as open("data.csv", "r") to csv.reader for testing
    # read and process line by line don't read into list
    df_new = pd.read_csv(csvfile, names=flow_fields)

In [4]:
df_worms_mask = df_new['anomaly'] != 'Benign'
df_worm = df_new.loc[df_worms_mask]

Scaler for data standartization

In [5]:
def do_scl(df_num, cols):
    print("Original values:\n", df_num)

    scaler = RobustScaler()
    scaler_temp = scaler.fit_transform(df_num)

    std_df = pd.DataFrame(scaler_temp, columns =cols)

    #print("\nScaled values:\n", std_df)

    return std_df

cat_cols = ['ip_protocol']

Process standatrization and normalization primitive

In [7]:
def process(dataframe):
    df_num = dataframe.drop(cat_cols, axis=1)
    num_cols = df_num.columns
    scaled_df = do_scl(df_num, num_cols)

    dataframe.drop(labels=num_cols, axis="columns", inplace=True)
    dataframe[num_cols] = scaled_df[num_cols]

    #print("Before encoding:")
    #print(dataframe['ip_protocol'])
    
    #uncomment for categorical features
    #dataframe = pd.get_dummies(dataframe, columns = ['ip_protocol'])

    #print("\nColumns after encoding:")
    #print(dataframe.filter(regex='^protocol_type_'))
    
    return dataframe

Drop not necessary columns and process scaling

In [8]:
df_test =df_worm.drop(['src_ip', 'dst_ip','l7_proto','anomaly','notneeded'] ,axis=1)

In [9]:
df_test.reset_index(inplace=True)
df_test.drop(['index'],axis=1, inplace=True)

Scale fetures

In [10]:
scaled_test = process(df_test)

Original values:
          src_port  dst_port  in_bytes  out_bytes  in_pkts  out_pkts  \
0           43025        25     41290       2080       48        24   
1           64923        80       994        256       10         6   
2            9022       111       552        336       10         8   
3            1752        25      3022       1636       20        20   
4           53967       111       168          0        2         0   
...           ...       ...       ...        ...      ...       ...   
2786840     46307        22      4854       5812       32        36   
2786841     50032      4433   2334879     897197      553       303   
2786842      4444     49160    171528  209050328     3009      3958   
2786843      4433     50032    303896  151306895     4974     13807   
2786844     49160      4444  40102320      37280      763       590   

         tcp_flags  duration  label  
0               27       803      1  
1               19       189      1  
2              

Split data in to training and validation sets

In [11]:
y_test_live = scaled_test['label'].values
y_test_live = y_test_live.astype('int')

X_test_live = scaled_test.drop(['label'], axis=1)

n_estimators=1500 - the higher the value, the longer the training takes

Start learning process
These values can be tuned for best performance:
'random_state=47, contamination=0.01,n_estimators=1000'

Start prediction. Trained model know only x_train data

In [12]:
# Load trained model
from pickle import load
with open("IsolationForestModel_100.pkl", "rb") as f:
    clf = load(f)

In [13]:
predict_test_live = clf.predict(X_test_live)

Normalize predicted values to source values for testing prediction:  0- is normal behviour, 1 - is anomaly

In [14]:
predict_test_live[predict_test_live == 1] = 0
predict_test_live[predict_test_live == -1] = 1

Validate predicted values to known values and calculate accuracy

In [15]:
test_accuracy_live = metrics.accuracy_score(y_test_live,predict_test_live)

Print results for accuracy

In [16]:
print("Predict Accuracy " + "IsolationForestClassifier" + " {}  Predict Accuracy ".format(test_accuracy_live*100))
test_accuracy_live

Predict Accuracy IsolationForestClassifier 99.99425874061887  Predict Accuracy 


0.9999425874061887